# 5.9 含并行连结的网络（GoogLeNet）

In [10]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.__version__)
print(device)

1.2.0
cuda


## 5.9.1 Inception 块

In [11]:
class Inception(nn.Module):
    # c1 - c4为每条线路里的层的输出通道数
    def __init__(self, in_c, c1, c2, c3, c4):
        super(Inception, self).__init__()
        # 线路1，单1 x 1卷积层
        self.p1_1 = nn.Conv2d(in_c, c1, kernel_size=1)
        # 线路2，1 x 1卷积层后接3 x 3卷积层
        self.p2_1 = nn.Conv2d(in_c, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1 x 1卷积层后接5 x 5卷积层
        self.p3_1 = nn.Conv2d(in_c, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3 x 3最大池化层后接1 x 1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_c, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        return torch.cat((p1, p2, p3, p4), dim=1)  # 在通道维上连结输出

## 5.9.2 GoogLeNet模型

In [12]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [13]:
b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [14]:
b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [15]:
b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [16]:
b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   d2l.GlobalAvgPool2d())

In [17]:
net = nn.Sequential(b1, b2, b3, b4, b5, d2l.FlattenLayer(), nn.Linear(1024, 10))
X = torch.rand(1, 1, 96, 96)
for blk in net.children(): 
    X = blk(X)
    print('output shape: ', X.shape)

output shape:  torch.Size([1, 64, 24, 24])
output shape:  torch.Size([1, 192, 12, 12])
output shape:  torch.Size([1, 480, 6, 6])
output shape:  torch.Size([1, 832, 3, 3])
output shape:  torch.Size([1, 1024, 1, 1])
output shape:  torch.Size([1, 1024])
output shape:  torch.Size([1, 10])


## 5.9.3 获取数据和训练模型

In [9]:
batch_size = 128
# 如出现“out of memory”的报错信息，可减小batch_size或resize
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 1.2133, train acc 0.524, test acc 0.804, time 45.7 sec
epoch 2, loss 0.2134, train acc 0.843, test acc 0.853, time 58.9 sec
epoch 3, loss 0.1151, train acc 0.871, test acc 0.880, time 58.9 sec
epoch 4, loss 0.0759, train acc 0.887, test acc 0.887, time 59.1 sec
epoch 5, loss 0.0547, train acc 0.900, test acc 0.884, time 59.3 sec


## My_Googlenet

In [38]:
class inception(nn.Module):
    def __init__(self,in_c,c1,c2,c3,c4):
        super(inception,self).__init__()
        self.p1=nn.Conv2d(in_c,c1,(1,1))
        self.p2_1=nn.Conv2d(in_c,c2[0],(1,1))
        self.p2_2=nn.Conv2d(c2[0],c2[1],(3,3),1,1)
        self.p3_1=nn.Conv2d(in_c,c3[0],(1,1))
        self.p3_2=nn.Conv2d(c3[0],c3[1],(5,5),1,2)
        self.p4_1=nn.MaxPool2d((3,3),1,1)
        self.p4_2=nn.Conv2d(in_c,c4,(1,1))
    def forward(self,x):
        t1=F.relu(self.p1(x))
        t2=F.relu(self.p2_2(F.relu(self.p2_1(x))))
        t3=F.relu(self.p3_2(F.relu(self.p3_1(x))))
        t4=F.relu(self.p4_2(self.p4_1(x)))
        return torch.cat((t1,t2,t3,t4),1)

In [45]:
block1=nn.Sequential(nn.Conv2d(1,64,7,2,3),
                     nn.ReLU(),
                     nn.MaxPool2d(3,2,1)
                    )
block2=nn.Sequential(nn.Conv2d(64,64,1),
                     nn.Conv2d(64,192,3,1,1),
                     nn.MaxPool2d(3,2,1)
)
block3=nn.Sequential(inception(192,64,(96,128),(16,32),32),
                    inception(256,128,(128,192),(32,96),64),
                    nn.MaxPool2d(3,2,1))
block4=nn.Sequential(inception(480,192,(96,208),(16,48),64),
                     inception(512,160,(112,224),(24,64),64),
                     inception(512,128,(128,256),(24,64),64),
                     inception(512,112,(144,288),(32,64),64),
                     inception(528,256,(160,320),(32,128),128),
                     nn.MaxPool2d(3,2,1)
                    )
block5=nn.Sequential(inception(832,256,(160,320),(32,128),128),
                     inception(832,384,(192,384),(48,128),128),
                     d2l.GlobalAvgPool2d()
)
net=nn.Sequential(block1,block2,block3,block4,block5,d2l.FlattenLayer(),nn.Linear(1024,10))


In [46]:
X = torch.rand(1, 1, 96, 96)
for blk in net.children(): 
    X = blk(X)
    print('output shape: ', X.shape)

output shape:  torch.Size([1, 64, 24, 24])
output shape:  torch.Size([1, 192, 12, 12])
output shape:  torch.Size([1, 480, 6, 6])
output shape:  torch.Size([1, 832, 3, 3])
output shape:  torch.Size([1, 1024, 1, 1])
output shape:  torch.Size([1, 1024])
output shape:  torch.Size([1, 10])


In [43]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cuda
epoch 1, loss 1.5029, train acc 0.404, test acc 0.733, time 28.5 sec
epoch 2, loss 0.2456, train acc 0.815, test acc 0.840, time 28.4 sec
epoch 3, loss 0.1201, train acc 0.863, test acc 0.860, time 28.8 sec
epoch 4, loss 0.0788, train acc 0.881, test acc 0.876, time 28.9 sec
epoch 5, loss 0.0572, train acc 0.892, test acc 0.885, time 28.9 sec
